In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import os, re, sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from yuval_module.paper_clusterer import PaperClusterer
from yuval_module.paper_source import PaperSource

from sklearn.feature_extraction.text import TfidfVectorizer

import py_3.sim_matrix_3 as sim_matrix_3
import py_3.lr_model_3 as lr_model_3
import py_3.db_scan_3 as db_scan_3
import py_4.get_mesh_vec as get_mesh_vec

import metric_eval_2

%matplotlib inline


import utils
from utils import PROJECT_ROOT, DATA_PATH

FILE = "enriched_labeled_dataset_large.json"

In [3]:
PROJECT_ROOT + DATA_PATH + FILE

'/home/ubuntu/Proj/AYP/data/labeled_data/enriched_labeled_dataset_large.json'

In [4]:
if os.path.exists(PROJECT_ROOT + DATA_PATH + FILE):
    print("READING FROM LOCAL")
    if FILE.split(".")[1] == "json":
        df = pd.read_json(PROJECT_ROOT + DATA_PATH + FILE)
    else:
        df = pd.read_csv(PROJECT_ROOT + DATA_PATH + FILE)
    ps = PaperSource()
else:
    print("PULLING FROM S3")
    ps = sim_matrix_3.load_dataset("enriched_labeled_dataset")
    df = ps.get_dataset()

df.drop(columns=["last_author_country"],inplace=True)
df.rename(columns={'ORG_STATE':'last_author_country'},inplace=True)

print("FILE PULLED")

READING FROM LOCAL
FILE PULLED


In [6]:
def mesh2int(row: list, dict_meshtoint: dict) -> list:
    '''
    Helper function for get_mesh_gram_freq. Turns mesh terms into label equivalent

        :param pd.Series row - list of mesh terms for given row
        :param dict dict_meshtoint - dict of mesh terms to their int labels
        :return None - if row is empty
                list - list of labels
    '''
    if row is None:
        return None
    lst = []
    for value in row:
        lst.append(dict_meshtoint[value])
    return lst

In [7]:
all_mesh = df['mesh'].values.tolist()
set_mesh = set()
for mesh in all_mesh:
    if mesh is not None:
        set_mesh.update(mesh)
# dict_inttomesh = {i:mesh for i,mesh in enumerate(set_mesh)}
dict_meshtoint = {mesh:i for i,mesh in enumerate(set_mesh)}
df_mesh = pd.Series([mesh2int(row, dict_meshtoint) for row in df['mesh'].values])

In [32]:
corpus = ["".join(str(terms)).strip("[").strip("]") if terms is not None else "" for terms in list(df_mesh.values)]

In [48]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(corpus)

In [50]:
X[0]

<1x1605611 sparse matrix of type '<class 'numpy.float64'>'
	with 21 stored elements in Compressed Sparse Row format>